In [1]:
### APO Structure creation ###

from SBLMDCOVDOCK import SBLSettings

settings = SBLSettings.GROMACS_Settings()

import pandas as pd
import os

In [2]:
# collect APO structures 

# AMPC - https://journals.asm.org/doi/10.1128/AAC.02073-20
ampc = "6T3D"
# KPC-2 - BLDB: http://dx.doi.org/10.1021/ACS.JMEDCHEM.7B00158
kpc2 = "5UL8"
# OXA-10 - BLDB: https://www.pnas.org/doi/full/10.1073/pnas.241442898
oxa10 = "1K55"

structures = pd.DataFrame({"PDBID": [ampc, kpc2, oxa10], 
                           "Name": ["AmpC", "KPC2", "OXA10"]})



In [3]:
# Download structures 
# only Chani A

from pdbtools import *

for pdbcode in structures.PDBID:
    output_path = os.path.join(settings.structures_input, pdbcode + ".pdb")
    !pdb_fetch {pdbcode} | pdb_selchain -A > {output_path}


In [4]:
# Rename KCX from hetatom to atom


for pdbcode in structures.PDBID:
    print(pdbcode)
    output_path = os.path.join(settings.structures_input, pdbcode + ".pdb")

    with open(output_path, "r") as f:
        lines = f.readlines()
        
        for idx,line in enumerate(lines):
            # print(line)
            split = line.split(' ')
            # remove empty strings
            split = list(filter(None, split))
            # print(line)
            try:
                if split[3] == "KCX":
                    # print(line)
                    lines[idx] = line.replace("HETATM", "ATOM  ")
                    print(idx, line)
            except:
                pass
            # print(line)

    # output_path = output_path.replace(".pdb", "_atom.pdb")
    with open(output_path, "w+") as f:
        for line in lines:
            f.write(line)



6T3D
5UL8
1K55
1579 HETATM  371  N   KCX A  70      17.691  11.113  35.699  1.00 11.54           N  

1580 ANISOU  371  N   KCX A  70     1456   1528   1400   -113   -116    -26       N  

1581 HETATM  372  CA  KCX A  70      16.480  11.840  35.346  1.00 11.83           C  

1582 ANISOU  372  CA  KCX A  70     1508   1530   1457    -85   -135     15       C  

1583 HETATM  373  CB  KCX A  70      15.242  10.924  35.339  1.00 11.99           C  

1584 ANISOU  373  CB  KCX A  70     1543   1587   1423   -146    -88     97       C  

1585 HETATM  374  CG  KCX A  70      15.278   9.892  34.211  1.00 12.18           C  

1586 ANISOU  374  CG  KCX A  70     1618   1565   1442   -326   -205     43       C  

1587 HETATM  375  CD  KCX A  70      13.995   9.081  34.069  1.00 12.83           C  

1588 ANISOU  375  CD  KCX A  70     1475   1884   1512   -240   -156      5       C  

1589 HETATM  376  CE  KCX A  70      14.008   8.226  32.789  1.00 14.43           C  

1590 ANISOU  376  CE  KCX A 

In [5]:
# Create KCX pdb
for pdbcode in structures.PDBID:
    output_path = os.path.join(settings.structures_input, pdbcode + ".pdb")
    !pdb_selresname -KCX {output_path} | pdb_selchain -A > {output_path.replace(".pdb", "_KCX.pdb")}


In [6]:
## FIX PDB with WHATIF server:https://swift.cmbi.umcn.nl/servers/html/model.html
AMPCfixed = "fixed-2"
### TODO Find a correct way to fix the PDB file

In [7]:
# Clean structures from APO - remove water and hetatoms
for pdbcode in structures.PDBID:
    if pdbcode == "6T3D":
        input_path = os.path.join(settings.structures_input, AMPCfixed + ".pdb")
    else:
        input_path = os.path.join(settings.structures_input, pdbcode + ".pdb")
    print(input_path)
    cleaned_path = os.path.join(settings.structures_input, "APO_" + pdbcode + ".pdb")
    print(cleaned_path)
    !pdb_delhetatm {input_path} > {cleaned_path}

start_structures/APO/fixed-2.pdb
start_structures/APO/APO_6T3D.pdb
start_structures/APO/5UL8.pdb
start_structures/APO/APO_5UL8.pdb
start_structures/APO/1K55.pdb
start_structures/APO/APO_1K55.pdb


In [8]:
# view protonation states - only saves to the working dir
import propka

for pdbcode in structures.PDBID:
    print(pdbcode)
    cleaned_path = os.path.join(settings.structures_input, "APO_" + pdbcode + ".pdb")
    !propka3 {cleaned_path} -o 7.4 

6T3D
Missing atoms or failed protonation for ARG  14 A (ARG) -- please check the structure
Group (ARG) for    88-  CZ    14-ARG (A) [  -4.132   -3.038    0.867] C
Expected 8 interaction atoms for acids, found:
                89- NH1    14-ARG (A) [  -3.747   -2.699    2.093] N
                90- NH2    14-ARG (A) [  -3.487   -4.014    0.232] N
                87-  NE    14-ARG (A) [  -5.112   -2.393    0.244] N
                 0- HH1    14-ARG (A) [  -3.229   -3.330    2.688] H
                 0-HH21    14-ARG (A) [  -2.728   -4.499    0.689] H
                 0-HH22    14-ARG (A) [  -3.757   -4.270   -0.707] H
                 0-  HE    14-ARG (A) [  -5.261   -2.582   -0.737] H
Expected 3 interaction atoms for bases, found:
                89- NH1    14-ARG (A) [  -3.747   -2.699    2.093] N
                90- NH2    14-ARG (A) [  -3.487   -4.014    0.232] N
                87-  NE    14-ARG (A) [  -5.112   -2.393    0.244] N
Missing atoms or failed protonation for GLN 120 A (BB

In [9]:
import pdb2pqr

import subprocess

# Define the command as a list of strings



for pdbcode in structures.PDBID:
    cleaned_path = os.path.join(settings.structures_input, "APO_" + pdbcode + ".pdb")

    command = [
        "pdb2pqr",
        "--ff=AMBER",
        "--with-ph=7.4",
        "--titration-state-method=propka",
        cleaned_path,
        "-ffout=AMBER",
        cleaned_path.replace(".pdb", "_prot74.pqr"),
        "--pdb-output",
        cleaned_path.replace(".pdb", "_prot74.pdb"),
    ]

    subprocess.run(command, check=True)




INFO:PDB2PQR v3.6.1: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: start_structures/APO/APO_6T3D.pdb
ERROR:Error parsing line: invalid literal for int() with base 10: ''
ERROR:<CRYST1 1000.000 1000.000 1000.000  90.00  90.00  90.00 P 1>
ERROR:Truncating remaining errors for record type:CRYST1

ERROR:['CRYST1']
INFO:Setting up molecule.
INFO:Created biomolecule object with 358 residues and 2799 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 1 missing atoms in biomolecule

In [10]:
from SBLMDCOVDOCK.md_setuptools import parse_propka_output

for pdbcode in structures.PDBID:
    print(pdbcode)
    
    cleaned_path = os.path.join(settings.structures_input, "APO_" + pdbcode + ".pdb")


    protonation_states = parse_propka_output(cleaned_path.replace(".pdb", ".pka"), pH=7.4)
    protonation_states = pd.DataFrame.from_dict(protonation_states, orient="index")
    print(protonation_states)


6T3D
            0
(ASP, 10)   0
(ASP, 47)   0
(ASP, 76)   0
(ASP, 87)   0
(ASP, 123)  0
(ASP, 130)  0
(ASP, 217)  0
(ASP, 229)  0
(ASP, 242)  0
(ASP, 264)  0
(ASP, 275)  0
(ASP, 281)  0
(ASP, 288)  0
(ASP, 351)  0
(GLU, 21)   0
(GLU, 61)   0
(GLU, 82)   0
(GLU, 95)   0
(GLU, 171)  0
(GLU, 195)  0
(GLU, 196)  0
(GLU, 219)  0
(GLU, 228)  0
(GLU, 245)  0
(GLU, 272)  0
(GLU, 333)  0
(HIS, 13)   0
(HIS, 108)  0
(HIS, 186)  0
(HIS, 210)  0
(HIS, 314)  0
(LYS, 24)   1
(LYS, 37)   1
(LYS, 50)   1
(LYS, 67)   1
(LYS, 91)   1
(LYS, 224)  1
(LYS, 239)  1
(LYS, 315)  1
(LYS, 342)  1
(ARG, 80)   1
(ARG, 148)  1
(ARG, 177)  1
(ARG, 232)  1
(ARG, 258)  1
(ARG, 349)  1
5UL8
            0
(ASP, 39)   0
(ASP, 50)   0
(ASP, 92)   0
(ASP, 131)  0
(ASP, 157)  0
(ASP, 163)  0
(ASP, 176)  0
(ASP, 179)  0
(ASP, 209)  0
(ASP, 228)  0
(ASP, 233)  0
(ASP, 246)  1
(ASP, 271)  0
(ASP, 272)  0
(GLU, 31)   0
(GLU, 37)   0
(GLU, 63)   0
(GLU, 64)   0
(GLU, 110)  0
(GLU, 121)  0
(GLU, 141)  0
(GLU, 166)  0
(GLU, 168)

In [ ]:
KCX_note = """; KCX residue topology MODIFIED by Alexi on 31/07/2023 """

KCX_itp_atoms = """

[ atoms ]

;   nr  type  resi  res  atom  cgnr     charge      mass       ; qtot   bond_type

     1   N     1   KCX     N    1    -0.903800     14.01000 ; qtot -0.904

     2   CT     1   KCX    CA    2     0.127500     12.01000 ; qtot -0.776

     3   CT     1   KCX    CB    3    -0.117400     12.01000 ; qtot -0.894

     4   CT     1   KCX    CG    4    -0.079400     12.01000 ; qtot -0.973

     5   cT     1   KCX    CD    5    -0.092400     12.01000 ; qtot -1.065

     6   CT     1   KCX    CE    6     0.090000     12.01000 ; qtot -0.975

     7    N     1   KCX    NZ    7    -0.601900     14.01000 ; qtot -1.577 

     8    C     1   KCX     C    8     0.641100     12.01000 ; qtot -0.936

     9    O     1   KCX     O    9    -0.559000     16.00000 ; qtot -1.495

    10    C     1   KCX    CX   10     0.989600     12.01000 ; qtot -0.506

    11   OH     1   KCX   OXT   11    -0.607100     16.00000 ; qtot -1.113

    12   O2     1   KCX   OQ1   12    -0.843300     16.00000 ; qtot -1.956

    13   O2     1   KCX   OQ2   13    -0.843300     16.00000 ; qtot -2.799

    14   H     1   KCX     H   14     0.363800      1.00800 ; qtot -2.436

    15   H     1   KCX   HN2   15     0.363800      1.00800 ; qtot -2.072

    16   H1     1   KCX    HA   16     0.091700      1.00800 ; qtot -1.980

    17   HC     1   KCX   HB2   17     0.055200      1.00800 ; qtot -1.925

    18   HC     1   KCX   HB3   18     0.055200      1.00800 ; qtot -1.870

    19   HC     1   KCX   HG2   19     0.037700      1.00800 ; qtot -1.832

    20   HC     1   KCX   HG3   20     0.037700      1.00800 ; qtot -1.794

    21   HC     1   KCX   HD2   21     0.036200      1.00800 ; qtot -1.758

    22   HC     1   KCX   HD3   22     0.036200      1.00800 ; qtot -1.722

    23   H1     1   KCX   HE2   23     0.022200      1.00800 ; qtot -1.700

    24   H1     1   KCX   HE3   24     0.022200      1.00800 ; qtot -1.678

    25   H     1   KCX    HZ   25     0.241500      1.00800 ; qtot -1.436

    26   HO     1   KCX   HXT   26     0.436000      1.00800 ; qtot -1.000

"""
KCX_itp_partial_charges = [-0.9038,0.1275,-0.1174,-0.0794,-0.0924,0.09,-0.6019,0.6411,-0.559,0.9896,-0.6071,-0.8433, -0.8433,0.3638,0.3638,0.0917,0.0552,0.0552,0.0377,0.0377,0.0362,0.0362,0.0222,0.0222,0.2415,0.436]
print(sum(KCX_itp_partial_charges))
print(sum(KCX_itp_partial_charges) -- 1.0)


-1.0000000000000002
-2.220446049250313e-16


In [ ]:
# with open("KCXparams_GMX.itp", "r") as input_file:
#     atomtypes_section = False
#     for line in input_file:
#         line = line.strip()
#         if "[ atomtypes ]" in line:
#             atomtypes_section = True
#             continue
#         if atomtypes_section and line and not line.startswith(";"):
#             columns = line.split()
#             print(f"{columns[0]:<8s}{columns[2]:<10s}{columns[3]:<10s}A   {columns[5]:<10s}{columns[6]}")
#         if atomtypes_section and (not line or line.startswith(";")):
#             break

In [ ]:
# ### Topology found in https://mailman-1.sys.kth.se/pipermail/gromacs.org_gmx-users/2016-January/103280.html
# not using this, writing my own

In [ ]:
# ADD KCX to topology
### https://manual.gromacs.org/current/how-to/topology.html#
break # this will overide the changes TOOD, make changes a function
# Copy amber99 topology to working directory -
### Modifying a force field is best done by making a full copy of the installed forcefield directory and residuetypes.dat into your local working directory:
#Then, modify those local copies as above. pdb2gmx will then find both the original and modified version and you can choose the modified version interactively from the list, or if you use the pdb2gmx -ff option the local version will override the system version.
gmxtop = "/usr/local/gromacs/share/gromacs/top"
command = f"cp -r {gmxtop}/residuetypes.dat {gmxtop}/amber99sb-ildn.ff ."

os.system(command)



SyntaxError: 'break' outside loop (350209189.py, line 3)

In [ ]:
#use BioBB Ligand Parameterisation to generate the topology and parameters for the ligand
# manually adapt them to rtp topology file:
KCX_rtp = """
; Added by Alexi 31/07/23
[ KCX ]
 [ atoms ]
     N    N           -0.34790     1
     H    H            0.27470     2
    CA    CT          -0.24000     3
    HA    H1           0.14260     4
    CB    CT          -0.00940     5
   HB1    HC           0.03620     6
   HB2    HC           0.03620     7
    CG    CT           0.01870     8
   HG1    HC           0.01030     9
   HG2    HC           0.01030    10
    CD    CT          -0.04790    11
   HD1    HC           0.06210    12
   HD2    HC           0.06210    13
    CE    CT          -0.01430    14
   HE1    HP           0.11350    15
   HE2    HP           0.11350    16
    NZ    N           -0.38540    17
   HZ1    H            0.34000    18
     C    C            0.73410    19
     O    O           -0.58940    20
    CX    C            0.73410    21
   OQ1    O2          -0.81880    22 ;taken from GLU
   OQ2    O2          -0.81880    23 ;taken from GLU

 [ bonds ]
     N     H
     N    CA
    CA    HA
    CA    CB
    CA     C
    CB   HB1
    CB   HB2
    CB    CG
    CG   HG1
    CG   HG2
    CG    CD
    CD   HD1
    CD   HD2
    CD    CE
    CE   HE1
    CE   HE2
    CE    NZ
    NZ   HZ1
     C     O
    -C     N
    NZ    CX
    CX    OQ1
    CX    OQ2

 [ impropers ]
    -C    CA     N     H
    CA    +N     C     O
    N     O2     C    O2 ;Adapted from GLU - listed as X  O2  C  O2 in ffbonded.itp
    C     CT     N    H ;listed as X  CT  N  H in ffbonded.itp
    """
# Check partial charges
KCX_charges = [-0.3479,
 0.2747,
 -0.24,
 0.1426,
 -0.0094,
 0.0362,
 0.0362,
 0.0187,
 0.0103,
 0.0103,
 -0.0479,
 0.0621,
 0.0621,
 -0.0143,
 0.1135,
 0.1135,
 -0.3854,
 0.34,
 0.7341,
 -0.5894,
 0.7341,
 -0.8188,
 -0.8188]



KCX_charges = [-0.3479,
 0.2747,
 -0.24,
 0.1426,
 -0.0094,
 0.0362,
 0.0362,
 0.0187,
 0.0103,
 0.0103,
 -0.0479,
 0.0621,
 0.0621,
 -0.0143,
 0.1135,
 0.1135,
 -0.3854,
 0.34,
 0.7341,
 -0.5894,
 0.7342,
 -1.0271,
 -1.0271]

print(sum(KCX_charges))
print(sum(KCX_charges)--1)

import numpy as np
KCX_charges = np.array(KCX_charges)
KCX_charges = KCX_charges*(-1/KCX_charges.sum())
print(KCX_charges.round(5))

-1.0
0.0
[-0.3479  0.2747 -0.24    0.1426 -0.0094  0.0362  0.0362  0.0187  0.0103
  0.0103 -0.0479  0.0621  0.0621 -0.0143  0.1135  0.1135 -0.3854  0.34
  0.7341 -0.5894  0.7342 -1.0271 -1.0271]


In [ ]:
KCX_hdb = """
; Added by Alexi 31/07/2023
KCX	7
1	1	H	N	-C	CA	
1	5	HA	CA	N	CB	C	
2	6	HB	CB	CA	CG	
2	6	HG	CG	CB	CD	
2	6	HD	CD	CG	CE	
2	6	HE	CE	CD	NZ	
1	4	HZ	NZ	CE	CD	;changed from 3->1
"""

In [ ]:
for pdbcode in structures.PDBID:
    input_path = os.path.join(settings.structures_input, "APO_" + pdbcode + ".pdb")

    output_path = os.path.join(settings.structures_output, "APO_" + pdbcode + ".gro")
    topo_path = os.path.join(settings.structures_output, "APO_" + pdbcode + ".top")
    print(input_path)
    print(output_path)
    print(topo_path)
    command = ["gmx", "pdb2gmx", 
                    "-f", input_path, 
                    "-o", output_path, 
                    "-p", topo_path, "-water", "tip3p", 
                    "-ff", "amber99sb-ildn",
                    "-ignh",
                    "-inter"]
    # subprocess.run(command, check=True)
### THIS has to be done manually, because the program has interactive input
# need to add KCX to the topology file
# HIS set to HISE (HIE) when done non-interactively - is this correct?
# Check with _prot74.pdb - protons not recognised
# 6T3D only works with this method - everything set to default TODO Check this as we still set -ignh
#
# gmx pdb2gmx -f start_structures/APO/APO_1K55.pdb -o prod_structures/APO/APO_1K55.gro -p prod_structures/APO/APO_1K55.top -water tip3p -ff amber99sb-ildn -ignh -inter
# gmx pdb2gmx -f start_structures/APO/APO_5UL8.pdb -o prod_structures/APO/APO_5UL8.gro -p prod_structures/APO/APO_5UL8.top -water tip3p -ff amber99sb-ildn -ignh -inter
# gmx pdb2gmx -f start_structures/APO/APO_6T3D_prot74.pdb -o prod_structures/APO/APO_6T3D.gro -p prod_structures/APO/APO_6T3D.top -water tip3p -ff amber99sb-ildn -ignh

# TODO Check HIS states - Not accurate, taken from pdb2gmx -nointer
# TODO Check Protonation states - not complete in some cases
# TODO Check partial charges - fabricated

start_structures/APO/APO_6T3D.pdb
prod_structures/APO/APO_6T3D.gro
prod_structures/APO/APO_6T3D.top
start_structures/APO/APO_5UL8.pdb
prod_structures/APO/APO_5UL8.gro
prod_structures/APO/APO_5UL8.top
start_structures/APO/APO_1K55.pdb
prod_structures/APO/APO_1K55.gro
prod_structures/APO/APO_1K55.top


In [ ]:
KCX_angle_types = """"
[ angletypes ]
;  i    j    k  func       th0       cth
HP  CT  N            1   108.880    514.630 ; Added by Alexi 31/07/2023
N   C   O2           1   123.050    952.280 ; Added by Alexi 31/07/2023
"""
# need to add this to the topology file and the forcefield file 

,0
"(ASP, 55)",deprotonated
"(ASP, 93)",deprotonated
"(ASP, 105)",deprotonated
"(ASP, 151)",deprotonated
"(ASP, 240)",deprotonated
...,...
"(ARG, 109)",protonated
"(ARG, 125)",protonated
"(ARG, 131)",protonated
"(ARG, 160)",protonated


In [ ]:

input_path = os.path.join(settings.structures_output, test_pdb)
output_path = os.path.join(settings.structures_output, test_pdb.replace(".pdb", ".gro"))
topo_path = os.path.join(settings.structures_output, test_pdb.replace(".pdb", ".top"))

subprocess.run(["gmx", "pdb2gmx", 
                "-f", input_path, 
                "-o", output_path, 
                "-p", topo_path, "-water", "tip3p", 
                "-ff", "amber99sb-ildn",
                "-ignh",], 
                check=True)


Using the Amber99sb-ildn force field in directory amber99sb-ildn.ff

going to rename amber99sb-ildn.ff/aminoacids.r2b

going to rename amber99sb-ildn.ff/dna.r2b

going to rename amber99sb-ildn.ff/rna.r2b
Reading prod_structures/APO/0.15_80_10_pH7.4_APO_1K55.result.pdb...
Read '', 1928 atoms

Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.

There are 2 chains and 0 blocks of water and 244 residues with 1928 atoms

  chain  #res #atoms

  1 ' '    49    371  

  2 ' '   195   1557  

there were 91 atoms with zero occupancy and 1837 atoms with          occupancy unequal to one (out of 1928 atoms). Check your pdb file.

Reading residue database... (Amber99sb-ildn)

Processing chain 1 (371 atoms, 49 residues)

Identified residue SER21 as a starting terminus.

Identified residue PHE69 as a ending terminus.

Checking for duplicate atoms....

Generating any missing hydrogen atoms and/or adding termini.

Now there are 49 residues with 728 atoms
Chain time

                     :-) GROMACS - gmx pdb2gmx, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_2/Project/SBL_MD_CovDock
Command line:
  gmx pdb2gmx -f prod_structures/APO/0.15_80_10_pH7.4_APO_1K55.result.pdb -o prod_structures/APO/0.15_80_10_pH7.4_APO_1K55.result.gro -p prod_structures/APO/0.15_80_10_pH7.4_APO_1K55.result.top -water tip3p -ff amber99sb-ildn -ignh

Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb-ildn.ff/aminoacids.r2b
Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb-ildn.ff/dna.r2b
Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb-ildn.ff/rna.r2b
there were 91 atoms with zero occupancy and 1837 atoms with          occupancy unequal to one (out of 1928 atoms). Check your pdb file.
Opening force field file /usr/local/gromacs/share/gromacs/top/amber99sb-ildn.ff/atomtypes.atp


CompletedProcess(args=['gmx', 'pdb2gmx', '-f', 'prod_structures/APO/0.15_80_10_pH7.4_APO_1K55.result.pdb', '-o', 'prod_structures/APO/0.15_80_10_pH7.4_APO_1K55.result.gro', '-p', 'prod_structures/APO/0.15_80_10_pH7.4_APO_1K55.result.top', '-water', 'tip3p', '-ff', 'amber99sb-ildn', '-ignh'], returncode=0)

In [ ]:
# Prep simulations - set box size, solvate and ions

# use GMXAPI


